<a href="https://colab.research.google.com/github/sidharkal/RAG-/blob/main/Agentic_RAG_with_LangChain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Installing Dependencies

In [1]:
!pip install langchain langchain-openai faiss-cpu chromadb tiktoken pypdf langgraph
!pip install -U tavily-python langchain_community

In [2]:
!wget https://arxiv.org/pdf/1706.03762
!mkdir docs
!mv 1706.03762 docs/attention.pdf

--2025-02-22 09:40:39--  https://arxiv.org/pdf/1706.03762
Resolving arxiv.org (arxiv.org)... 151.101.131.42, 151.101.3.42, 151.101.67.42, ...
Connecting to arxiv.org (arxiv.org)|151.101.131.42|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2215244 (2.1M) [application/pdf]
Saving to: ‘1706.03762’

1706.03762          100%[===================>]   2.11M  --.-KB/s    in 0.09s   

2025-02-22 09:40:40 (24.0 MB/s) - ‘1706.03762’ saved [2215244/2215244]

mkdir: cannot create directory ‘docs’: File exists


In [3]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings.openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader,PyPDFLoader
from langchain_core.tools import tool
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, Tool, AgentType

In [4]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [5]:
# Load data from a pdf file
loader = PyPDFLoader("docs/attention.pdf")
documents = loader.load()

In [6]:
# Split data into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
docs = text_splitter.split_documents(documents)

In [7]:
# Create a vector store with embeddings
embeddings = OpenAIEmbeddings()
vector_store = FAISS.from_documents(docs, embeddings)


<ipython-input-7-657efb989347>:2: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()


In [8]:
# Initialize a retriever for querying the vector store
retriever = vector_store.as_retriever(search_type="similarity", search_k=5)

In [9]:
# Example: Weather API function
def get_weather(location: str):
    return f"The current weather in {location} is sunny and 25°C."

# Define the tool
weather_tool = Tool(
    name="Weather Tool",
    func=get_weather,
    description="Provides weather information for a given location.")

In [10]:
# Initialize the LLM
model  = ChatOpenAI(model="gpt-4")

# Create the Retrieval QA chain
retrieval_qa_chain = RetrievalQA.from_chain_type(
    llm=model,
    retriever=retriever,
    return_source_documents=True)

<ipython-input-10-7fed7849dcf3>:2: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  model  = ChatOpenAI(model="gpt-4")


In [11]:
# Combine tools and retrieval chain
tools = [
    Tool(
        name="Document Retrieval",
        func=lambda q: retrieval_qa_chain({"query": q})["result"],
        description="Retrieve knowledge from the document database."
    ),
    weather_tool
]

# Initialize the agent
agent = initialize_agent(
    tools=tools,
    llm=model,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True)


<ipython-input-11-31a882bdf602>:12: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


In [12]:
user_query = "what is agent in GEN AI"
response = agent.run(user_query)
print(f"response  : {response}")

<ipython-input-12-0043ccc831f7>:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = agent.run(user_query)




> Entering new AgentExecutor chain...
I need to find a document that explains what an agent is in the context of GEN AI.
Action: Document Retrieval
Action Input: agent in GEN AI

<ipython-input-11-31a882bdf602>:5: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  func=lambda q: retrieval_qa_chain({"query": q})["result"],



Observation: I'm sorry, but the context provided doesn't give information about an agent in GEN AI.
Thought:It seems like the document retrieval tool couldn't find any specific information about an agent in GEN AI. I could make an educated guess based on the general understanding of what an agent is in AI. 
Final Answer: In the context of AI, an agent is generally defined as an entity that perceives its environment and takes actions to achieve its goal. However, I couldn't find specific information about an agent in the context of GEN AI.

> Finished chain.
response  : In the context of AI, an agent is generally defined as an entity that perceives its environment and takes actions to achieve its goal. However, I couldn't find specific information about an agent in the context of GEN AI.


In [13]:
user_query = "can you explain me Transformer architecture"
response = agent.run(user_query)
print(f"response  : {response}")



> Entering new AgentExecutor chain...
The Transformer architecture is a complex concept in the field of artificial intelligence and machine learning. I will need to retrieve a document explaining it. 
Action: Document Retrieval
Action Input: Transformer architecture
Observation: The Transformer architecture is a model architecture used in neural sequence transduction models. It uses stacked self-attention and point-wise, fully connected layers for both the encoder and decoder. The Transformer relies entirely on an attention mechanism to draw global dependencies between input and output. This allows for significantly more parallelization and can lead to a high level of translation quality. It has been shown to generalize well to other tasks, such as English constituency parsing. The architecture is divided into two halves, with the encoder shown on the left and the decoder on the right.
Thought:I now have a detailed explanation of the Transformer architecture. 
Final Answer: The Trans